In [1]:
from implicit.evaluation import  *
from implicit.als import AlternatingLeastSquares as ALS
from implicit.bpr import BayesianPersonalizedRanking as BPR
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.utils import shuffle
from scipy.sparse import *
# import rec_util
pd.options.display.float_format = '{:.2f}'.format
import pandas as pd

In [2]:
song_meta = pd.read_json('./raw_data/song_meta.json')
train = pd.read_json('./raw_data/train.json')
val = pd.read_json('./raw_data/val.json')
genre = pd.read_json('./raw_data/genre_gn_all.json', typ = 'seriese')

In [3]:
song_meta.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


## id별 장르 구분

In [4]:
song_meta[['id','song_gn_dtl_gnr_basket','song_gn_gnr_basket']]

,id,song_gn_dtl_gnr_basket,song_gn_gnr_basket
0,0,[GN0901],[GN0900]
1,1,"[GN1601, GN1606]",[GN1600]
2,2,[GN0901],[GN0900]
3,3,"[GN1102, GN1101]",[GN1100]
4,4,"[GN1802, GN1801]",[GN1800]
...,...,...,...
707984,707984,[GN2001],[GN2000]
707985,707985,[GN0901],[GN0900]
707986,707986,"[GN0105, GN0101]",[GN0100]
707987,707987,"[GN1807, GN1801]",[GN1800]


In [5]:
song_meta['total_gn'] = song_meta['song_gn_dtl_gnr_basket'] + song_meta['song_gn_gnr_basket']

In [6]:
song_meta[['id','total_gn']]

,id,total_gn
0,0,"[GN0901, GN0900]"
1,1,"[GN1601, GN1606, GN1600]"
2,2,"[GN0901, GN0900]"
3,3,"[GN1102, GN1101, GN1100]"
4,4,"[GN1802, GN1801, GN1800]"
...,...,...
707984,707984,"[GN2001, GN2000]"
707985,707985,"[GN0901, GN0900]"
707986,707986,"[GN0105, GN0101, GN0100]"
707987,707987,"[GN1807, GN1801, GN1800]"


In [7]:
df = song_meta[['id','total_gn']].explode('total_gn')
# df1.explode('song_gn_dtl_gnr_basket')
df

,id,total_gn
0,0,GN0901
0,0,GN0900
1,1,GN1601
1,1,GN1606
1,1,GN1600
...,...,...
707987,707987,GN1801
707987,707987,GN1800
707988,707988,GN0601
707988,707988,GN0604


# 이진행렬 만들기

In [8]:
df1 = pd.get_dummies(df['total_gn']).max(level=0)

In [9]:
df1.shape

(707989, 249)

## 빠진 장르 확인

In [10]:
a = df1.columns.difference(['id']).tolist()

In [11]:
b = genre.index.tolist()

In [12]:
s = set(a)
temp3 = [x for x in b if x not in s]

In [13]:
temp3

['GN0305', 'GN1612', 'GN1709', 'GN2110', 'GN2604', 'GN3000']

In [14]:
genre_df = pd.DataFrame(genre).reset_index()
genre_df

,index,0
0,GN0100,발라드
1,GN0101,세부장르전체
2,GN0102,'80
3,GN0103,'90
4,GN0104,'00
...,...,...
249,GN2900,뮤지컬
250,GN2901,세부장르전체
251,GN2902,국내뮤지컬
252,GN2903,국외뮤지컬


In [15]:
genre_df[genre_df['index'] == 'GN3000']

,index,0
253,GN3000,크리스마스


In [16]:
tr_csr = csr_matrix(df1.values)

In [17]:
tr_csr

<707989x249 sparse matrix of type '<class 'numpy.uint8'>'
	with 2315254 stored elements in Compressed Sparse Row format>

In [18]:
# # 메모리 터짐
# from sklearn.metrics import pairwise_distances
# pairwise_matrix = pairwise_distances(tr_csr, tr_csr, n_jobs=-1)

In [19]:
# model 1

In [20]:
als_model = ALS(factors=256, regularization=0.08)

## .T 한거

In [19]:
als_model.fit(tr_csr.T * 15.0)

## .T 안한거

In [13]:
als_model.fit(tr_csr * 15.0)

## .T 안한거, 15안 곱한거
- 노래 100개 먼저 채우기

In [21]:
als_model.fit(tr_csr * 32)

In [25]:
def similar_songs(song_list):
    ls = []
    for song in song_list:
        ls += als_model.similar_items(song, N=100)[:int(100 / len(song_list)) + 1]
    return [i[0] for i in ls][:100]

# 노래 먼저 100개 채우기

In [67]:
val_song_100 = val.copy()

In [70]:
from tqdm import tqdm
tqdm.pandas()

In [77]:
val_song_100['songs_predicted'] = val_song_100['songs'].progress_apply(similar_songs)



  0%|          | 0/23015 [00:00<?, ?it/s]

  0%|          | 2/23015 [00:01<3:47:16,  1.69it/s]

  0%|          | 4/23015 [00:01<3:14:13,  1.97it/s]

  0%|          | 5/23015 [00:02<3:42:36,  1.72it/s]

  0%|          | 6/23015 [00:02<3:18:52,  1.93it/s]

  0%|          | 7/23015 [00:03<3:13:54,  1.98it/s]

KeyboardInterrupt: 

In [ ]:
val_song_100.to_json('./val_song_100.json')

In [64]:
val_song_100['songs'].iloc[5]

nan

In [58]:
val['songs'].iloc[:5].apply(len)

0    27
1     0
2    14
3    17
4     8
Name: songs, dtype: int64

In [57]:
val_song_100['songs'].iloc[:50]

0     [373313, 381089, 584911, 45263, 151080, 367453...
1                                                    []
2     [529437, 250084, 91573, 142777, 57149, 230079,...
3     [589668, 2059, 350869, 567759, 449088, 372018,...
4     [672718, 517501, 376505, 81712, 373836, 368397...
5                                                   NaN
6                                                   NaN
7                                                   NaN
8                                                   NaN
9                                                   NaN
10                                                  NaN
11                                                  NaN
12                                                  NaN
13                                                  NaN
14                                                  NaN
15                                                  NaN
16                                                  NaN
17                                              

In [72]:
len(result[0])

100

In [73]:
len(result[2])

100

In [ ]:
[:int(100 / len(song_list)) + 1]

In [48]:
song_meta.shape

(707989, 10)

In [63]:
for i in range(1,100):
    print(int(100 / i) + 1)

101
51
34
26
21
17
15
13
12
11
10
9
8
8
7
7
6
6
6
6
5
5
5
5
5
4
4
4
4
4
4
4
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


In [50]:
als_model.similar_items(707988)

[(178407, 0.28202176),
 (461104, 0.28202176),
 (518590, 0.28202176),
 (551086, 0.28202176),
 (254798, 0.28202176),
 (542532, 0.28202176),
 (431993, 0.28202176),
 (192726, 0.28202176),
 (585360, 0.28202176),
 (535448, 0.28202176)]

In [51]:
als_model.similar_items(0,N=100)

[(486774, 0.30090612),
 (423382, 0.30090612),
 (392043, 0.30090612),
 (252813, 0.30090612),
 (310474, 0.30090612),
 (560390, 0.30090612),
 (606898, 0.30090612),
 (175704, 0.30090612),
 (373673, 0.30090612),
 (52719, 0.30090612),
 (413399, 0.30090612),
 (236021, 0.30090612),
 (597865, 0.30090612),
 (117627, 0.30090612),
 (668088, 0.30090612),
 (445744, 0.30090612),
 (649735, 0.30090612),
 (485152, 0.30090612),
 (115412, 0.30090612),
 (93935, 0.30090612),
 (112261, 0.30090612),
 (409146, 0.30090612),
 (605098, 0.30090612),
 (300956, 0.30090612),
 (91299, 0.30090612),
 (204945, 0.30090612),
 (537226, 0.30090612),
 (92505, 0.30090612),
 (277138, 0.30090612),
 (93102, 0.30090612),
 (157105, 0.30090612),
 (376500, 0.30090612),
 (254721, 0.30090612),
 (470354, 0.30090612),
 (684698, 0.30090612),
 (706185, 0.30090612),
 (502879, 0.3009061),
 (80757, 0.3009061),
 (244308, 0.3009061),
 (404719, 0.3009061),
 (41629, 0.3009061),
 (245295, 0.3009061),
 (80741, 0.3009061),
 (501925, 0.3009061),
 (24

In [30]:
als_model.similar_items(0,N=150,)

[(308865, 0.3112559),
 (245105, 0.3112559),
 (447928, 0.3112559),
 (326168, 0.3112559),
 (222743, 0.3112559),
 (537163, 0.3112559),
 (52016, 0.3112559),
 (627252, 0.3112559),
 (183401, 0.3112559),
 (169502, 0.3112559),
 (361682, 0.3112559),
 (95617, 0.3112559),
 (544000, 0.3112559),
 (559722, 0.3112559),
 (83701, 0.3112559),
 (552604, 0.3112559),
 (246562, 0.3112559),
 (667109, 0.3112559),
 (298235, 0.3112559),
 (312848, 0.3112559),
 (623004, 0.3112559),
 (280502, 0.3112559),
 (220252, 0.3112559),
 (223779, 0.3112559),
 (570013, 0.3112559),
 (190944, 0.3112559),
 (490802, 0.3112559),
 (154505, 0.3112559),
 (43972, 0.3112559),
 (354354, 0.3112559),
 (607469, 0.3112559),
 (183204, 0.3112559),
 (83786, 0.3112559),
 (173309, 0.3112559),
 (490703, 0.3112559),
 (556567, 0.3112559),
 (579638, 0.3112559),
 (290995, 0.3112559),
 (659900, 0.3112559),
 (429692, 0.3112559),
 (214311, 0.3112559),
 (323902, 0.3112559),
 (547020, 0.3112559),
 (188209, 0.3112559),
 (484197, 0.3112559),
 (504363, 0.311

In [28]:
als_model.similar_items(707988)

[(454741, 0.3976742),
 (707988, 0.3976742),
 (105838, 0.39767417),
 (1969, 0.39767414),
 (529, 0.39767414),
 (390229, 0.39767414),
 (547565, 0.39767414),
 (311155, 0.3976741),
 (107999, 0.3976741),
 (336575, 0.3976741)]

In [47]:
test_ls = result[2000]
test_ls

[(84006, 4.7427106),
 (143195, 4.7427106),
 (72664, 4.7427106),
 (394746, 4.7427106),
 (379756, 4.7427106),
 (57304, 4.7427106),
 (654457, 4.7427106),
 (646450, 4.7427106),
 (91550, 4.7427106),
 (142600, 4.7427106),
 (405081, 4.7427106),
 (221871, 4.7427106),
 (603918, 4.7427106),
 (377574, 4.7427106),
 (237351, 4.7427106),
 (505966, 4.7427106),
 (215468, 4.7427106),
 (23422, 4.7427106),
 (479645, 4.7427106),
 (520730, 4.7427106),
 (141080, 4.7427106),
 (16510, 4.7427106),
 (420172, 4.7427106),
 (483874, 4.7427106),
 (322448, 4.7427106),
 (83457, 4.7427106),
 (98740, 4.7427106),
 (283222, 4.7427106),
 (35158, 4.7427106),
 (524544, 4.7427106),
 (10650, 4.7427106),
 (127714, 4.7427106),
 (381661, 4.7427106),
 (628495, 4.7427106),
 (588334, 4.7427106),
 (550271, 4.7427106),
 (292636, 4.7427106),
 (693945, 4.7427106),
 (518930, 4.7427106),
 (669701, 4.7427106),
 (353246, 4.7427106),
 (638296, 4.7427106),
 (511274, 4.7427106),
 (574790, 4.7427106),
 (345107, 4.7427106),
 (604405, 4.7427106)

In [48]:
test_ls.sort(key = lambda element : element[1],reverse=True)

In [49]:
test_ls

[(313836, 5.2222557),
 (336893, 5.2222557),
 (663556, 5.222255),
 (143438, 5.222255),
 (253466, 5.222255),
 (698779, 5.222255),
 (688050, 5.222255),
 (169158, 5.222255),
 (291014, 5.222255),
 (437699, 5.222255),
 (698304, 5.222255),
 (338827, 5.222255),
 (469256, 5.222255),
 (431313, 5.222255),
 (564627, 5.222255),
 (89331, 5.222255),
 (669003, 5.222255),
 (234921, 5.222255),
 (162558, 5.222255),
 (294339, 5.222255),
 (549764, 5.222255),
 (109656, 5.222255),
 (222172, 5.222255),
 (136479, 5.222255),
 (578917, 5.222255),
 (240092, 5.222255),
 (269296, 5.222255),
 (383324, 5.222255),
 (324881, 5.222255),
 (256712, 5.222255),
 (64762, 5.222255),
 (704154, 5.222255),
 (270761, 5.222255),
 (327107, 5.222255),
 (483781, 5.222255),
 (557567, 5.222255),
 (687419, 5.222255),
 (36241, 5.222255),
 (258652, 5.222255),
 (523929, 5.222255),
 (1326, 5.222255),
 (582065, 5.222255),
 (271812, 5.222255),
 (564548, 5.222255),
 (665699, 5.222255),
 (635315, 5.222255),
 (461699, 5.222255),
 (269817, 5.2222

In [111]:
list(dict.fromkeys([i[0] for i in test_ls]))

[228,
 122,
 226,
 119,
 28,
 123,
 121,
 120,
 150,
 22,
 249,
 88,
 57,
 3,
 127,
 143,
 63,
 23,
 56,
 5,
 151,
 247,
 128,
 154,
 31,
 25,
 89,
 124,
 4,
 144,
 21,
 20,
 55,
 73,
 116,
 231,
 85,
 69,
 80,
 65,
 81,
 84,
 66,
 76,
 176,
 36,
 83,
 129,
 148,
 24,
 2,
 41,
 6,
 173,
 75,
 199,
 233,
 223,
 153,
 74,
 53,
 211,
 222,
 205,
 134,
 198,
 38,
 232,
 33,
 32,
 246,
 192,
 245,
 162,
 161,
 97,
 90,
 91,
 101,
 118,
 95,
 82,
 117,
 58,
 159,
 26,
 237,
 146,
 158,
 100,
 248,
 78,
 102,
 141,
 14,
 30,
 225,
 94,
 236,
 142,
 47,
 13,
 139,
 18,
 227,
 160,
 42,
 229,
 125,
 136,
 230,
 140,
 106,
 207,
 79,
 235,
 107,
 19,
 203,
 132,
 93,
 109,
 27,
 67,
 98,
 110,
 185,
 114,
 186,
 71,
 1,
 147,
 188,
 167,
 17,
 130,
 35,
 72,
 99,
 126,
 149,
 216,
 10,
 215,
 105,
 16,
 9,
 70,
 189,
 64,
 197,
 138,
 37,
 180,
 133,
 15,
 190,
 196,
 60,
 131,
 224,
 135,
 87,
 7,
 115,
 234,
 171,
 50,
 77,
 8,
 51,
 92,
 104,
 202,
 195,
 166,
 178,
 54,
 200,
 164,
 108,
 17

In [18]:
df1

,id,GN0100,GN0101,GN0102,GN0103,GN0104,GN0105,GN0200,GN0201,GN0202,...,GN2802,GN2803,GN2804,GN2805,GN2806,GN2900,GN2901,GN2902,GN2903,GN9000
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707984,707984,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
707985,707985,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
707986,707986,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
707987,707987,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
als_model.similar_users(0)

[(0, 1.1300664),
 (2, 0.5172247),
 (286741, 0.48423234),
 (286377, 0.481705),
 (450071, 0.48153755),
 (5024, 0.48147815),
 (162596, 0.48119783),
 (177089, 0.481149),
 (256113, 0.48093686),
 (616432, 0.4804309)]

In [30]:
val.iloc[0]['songs']

[373313,
 151080,
 275346,
 696876,
 165237,
 525935,
 457812,
 371709,
 170292,
 438915,
 162569,
 161136,
 310892,
 608704,
 295701,
 127698,
 269427,
 592526,
 110696,
 368214,
 24386,
 104455,
 61612,
 151421,
 416023,
 320447,
 19606]

In [33]:
song_meta[song_meta['id'] == 373313]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,total_gn
373313,"[GN2207, GN1501, GN1506, GN1509]",20150101,Walt Disney Records The Legacy Collection: Pin...,2308104,[353277],I&#39;ve Got No Strings,"[GN1500, GN2200]",[Dickie Jones],373313,"[GN2207, GN1501, GN1506, GN1509, GN1500, GN2200]"


In [34]:
song_meta[song_meta['id'] == 475163]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,total_gn
475163,"[GN2207, GN1501, GN1506, GN1509]",20040929,마녀 배달부 키키 OST,2292012,[2796],傷心のキキ / Shoushinno Kiki (상심한 키키),"[GN1500, GN2200]",[Hisaishi Joe],475163,"[GN2207, GN1501, GN1506, GN1509, GN1500, GN2200]"


In [47]:
song_meta[song_meta['id'] == 386159]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,total_gn
386159,"[GN2207, GN1501, GN1506, GN1509]",20171113,엉뚱발랄 콩순이와 친구들,10110689,[2007534],할로윈 파티 (할로윈 파티 편),"[GN1500, GN2200]",[콩순이],386159,"[GN2207, GN1501, GN1506, GN1509, GN1500, GN2200]"


In [46]:
song_meta[song_meta['id'] == 319361]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,total_gn
319361,"[GN2207, GN1501, GN1506, GN1509]",20150701,Walt Disney Records The Legacy Collection: Cin...,2329099,"[45899, 38493, 40002, 560356]",A Visitor / Caught In A Trap / Lucifer / Feed ...,"[GN1500, GN2200]","[Mack David, Al Hoffman, Jerry Livingston, Oli...",319361,"[GN2207, GN1501, GN1506, GN1509, GN1500, GN2200]"


In [53]:
als_model.similar_users(0)

[(0, 1.1300664),
 (2, 0.5172247),
 (286741, 0.48423234),
 (286377, 0.481705),
 (450071, 0.48153755),
 (5024, 0.48147815),
 (162596, 0.48119783),
 (177089, 0.481149),
 (256113, 0.48093686),
 (616432, 0.4804309)]

In [54]:
als_model.similar_users(10000)

[(10000, 16.712063),
 (190964, 16.712057),
 (25135, 16.71205),
 (83819, 16.712038),
 (93248, 16.712023),
 (460243, 16.712017),
 (549436, 16.712015),
 (348799, 16.712011),
 (429918, 16.712004),
 (347758, 16.712)]